### XGBOOST 라이브러리를 활용한 머신러닝 학습, 예측

### 데이터 준비 및 라이브러리 불러오기

In [1]:
import numpy as np
import pandas as pd

In [8]:
# train 학습 데이터 셋 
# train를 train_x 학습용 입력, train_y 학습용 출력(예측대상)
# test_x 테스트 입력

train = pd.read_csv('../input/sample-data/train_preprocessed.csv')
train_x = train.drop(['target'], axis=1)
train_y = train['target']
test_x = pd.read_csv('../input/sample-data/test_preprocessed.csv')

In [14]:
train.columns

Index(['age', 'sex', 'height', 'weight', 'product', 'amount',
       'medical_info_a1', 'medical_info_a2', 'medical_info_a3',
       'medical_info_b1', 'medical_info_b2', 'medical_info_b3',
       'medical_info_c1', 'medical_info_c2', 'medical_keyword_1',
       'medical_keyword_2', 'medical_keyword_3', 'medical_keyword_4',
       'medical_keyword_5', 'medical_keyword_6', 'medical_keyword_7',
       'medical_keyword_8', 'medical_keyword_9', 'medical_keyword_10', 'year',
       'month', 'day', 'yearmonth', 'target'],
      dtype='object')

In [5]:
train_x.head(3)

,age,sex,height,weight,product,amount,medical_info_a1,medical_info_a2,medical_info_a3,medical_info_b1,...,medical_keyword_5,medical_keyword_6,medical_keyword_7,medical_keyword_8,medical_keyword_9,medical_keyword_10,year,month,day,yearmonth
0,50,1,166.445608,65.016732,9,7000000,134,202,1,11,...,0,1,0,1,0,0,2015,2,3,24182
1,68,0,164.334615,56.544217,0,7000000,438,263,3,14,...,0,0,1,1,0,0,2015,5,9,24185
2,77,1,167.462917,54.242267,2,6000000,313,325,1,18,...,0,1,0,1,0,0,2016,2,13,24194


In [7]:
train_y.head(3)

0    0
1    0
2    1
Name: target, dtype: int64

In [6]:
test_x.head(3)

,age,sex,height,weight,product,amount,medical_info_a1,medical_info_a2,medical_info_a3,medical_info_b1,...,medical_keyword_5,medical_keyword_6,medical_keyword_7,medical_keyword_8,medical_keyword_9,medical_keyword_10,year,month,day,yearmonth
0,49,1,187.431987,81.008363,1,1000000,302,212,1,10,...,0,1,0,1,0,0,2016,12,6,24204
1,79,1,171.632630,71.067812,6,2000,197,469,0,14,...,0,0,0,0,1,1,2016,9,3,24201
2,78,0,163.543983,64.032098,0,4000000,247,225,2,17,...,0,1,0,1,0,0,2015,4,10,24184


### Model 클래스 

* XGBoost 라이브러리를 사용하여 분류 모델을 구현한 클래스
* Model의 생성자, 학습메서드, 예측 메서드 정의

In [9]:
import xgboost as xgb

class Model:

    def __init__(self, params=None):
        self.model = None
        if params is None:
            self.params = {}
        else:
            self.params = params

    def fit(self, tr_x, tr_y):
        # params = {'objective': 'binary:logistic', 'silent': 1, 'random_state': 71}
        params = {'objective': 'binary:logistic', 'verbosity': 0, 'random_state': 71}
        params.update(self.params)
        num_round = 10
        dtrain = xgb.DMatrix(tr_x, label=tr_y)
        self.model = xgb.train(params, dtrain, num_round)

    def predict(self, x):
        data = xgb.DMatrix(x)
        pred = self.model.predict(data)
        return pred

### 모델 생성, 학습, 예측

In [11]:
# 모델의 하이퍼파라미터를 지정
params = {'param1': 10, 'param2': 100}

# Model 클래스를 정의
# Model 클래스는 fit로 학습하고 predict로 예측값 확률을 출력

# 모델 정의
model = Model(params)

# 학습 데이터로 모델 학습
model.fit(train_x, train_y)

# 테스트 데이터에 대해 예측 결과를 출력
pred = model.predict(test_x)
pred[0:10]

array([0.36924043, 0.14066012, 0.09131076, 0.05323898, 0.04907488,
       0.32397205, 0.47429284, 0.53221506, 0.48663282, 0.18739568],
      dtype=float32)

### 모델 예측 및 검증

In [12]:
# -----------------------------------
# 검증
# -----------------------------------
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold

# 학습 데이터 검증 데이터를 나누는 인덱스를 작성
# 학습 데이터를 4개로 나누고 그중 하나를 검증 데이터로 지정
kf = KFold(n_splits=4, shuffle=True, random_state=71)
tr_idx, va_idx = list(kf.split(train_x))[0]

# 학습 데이터와 검증 데이터로 구분
tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

# 모델 정의
model = Model(params)

# 학습 데이터에 이용하여 모델의 학습 수행
# 모델에 따라서는 검증 데이터를 동시에 제공하여 점수 모니터링
model.fit(tr_x, tr_y)

# 검증 데이터에 대해 예측하고 평가 수행
va_pred = model.predict(va_x)
score = log_loss(va_y, va_pred)
print(f'logloss: {score:.4f}')

logloss: 0.3009


### 교차 검증(crossvalidation)

In [13]:
# -----------------------------------
# 교차 검증(crossvalidation)
# -----------------------------------
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold

# 학습 데이터를 4개로 나누고 그 중 1개를 검증 데이터로 지정
# 분할한 검증 데이터를 바꾸어가며 학습 및 평가를 4회 실시
scores = []
kf = KFold(n_splits=4, shuffle=True, random_state=71)

for tr_idx, va_idx in kf.split(train_x):
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
    model = Model(params)
    model.fit(tr_x, tr_y)
    va_pred = model.predict(va_x)
    score = log_loss(va_y, va_pred)
    scores.append(score)

# 교차 검증의 평균 점수를 출력
print(f'logloss: {np.mean(scores):.4f}')

logloss: 0.2967
